In [1]:
!pip install pyswarms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 1.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 5.5 MB/s eta 0:00:00a 0:00:01


Import packages

In [2]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))
import pandas as pd
from configparser import ConfigParser
from repo.MyConfigParser import MyConfigParser
from repo.Optimization import SavantPSO
import numpy as np

# Read In

Import the config file and set the config dict. set some variables from the config dict

In [3]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/us/June 2024 Update/config ups us reg.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
optimization_start_date = config_dict['PROPHETSETTING']['optimization_start_date']
optimization_end_date = config_dict['PROPHETSETTING']['optimization_end_date']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

# Read data

Read the cuts df, hyper parameter table 

In [4]:
bdDf

NameError: name 'bdDf' is not defined

In [5]:

bdDf = pd.read_csv(base_path+'New_Cuts_df.csv',index_col=0) # read the new_cuts_df.csv file 
cuts_df = bdDf

# create a variable with the name of the pickle file
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' 
cuts_df['periods'] = periods # create the variable with the number of periods to forecast


In [6]:
cuts_df

,market,account_type,funnel,periods_for_acc_cal,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,input_mode,weekday_or_weekend,...,covid_year_dummy,anomaly_2021,generic_cost_split,competitor_cost_split,new_confirmed,power,growth,periods,cutoff_month,forecaster_name_list
0,US,Bing Ads,Brand,67,0.050,0.01,0.01,additive,multiplicative,additive,...,na,na,na,na,na,na,logistic,67.0,2024-04-25,US Bing Ads Brand.pkl
1,US,Bing Ads,Competitor,67,0.001,0.01,0.01,multiplicative,multiplicative,multiplicative,...,na,na,na,na,na,na,linear,67.0,2024-04-25,US Bing Ads Competitor.pkl
2,US,Bing Ads,Generic,67,0.001,0.10,0.01,additive,additive,multiplicative,...,na,na,na,na,na,na,logistic,67.0,2024-04-25,US Bing Ads Generic.pkl
3,US,Google AdWords,Brand,67,0.100,0.10,0.10,additive,additive,additive,...,na,na,na,na,na,na,logistic,67.0,2024-04-25,US Google AdWords Brand.pkl
4,US,Google AdWords,Competitor,67,0.001,0.01,0.10,multiplicative,additive,additive,...,na,na,na,na,na,na,linear,67.0,2024-04-25,US Google AdWords Competitor.pkl
5,US,Google AdWords,Generic,67,0.050,1.00,1.00,additive,multiplicative,multiplicative,...,na,na,na,na,na,na,linear,67.0,2024-04-25,US Google AdWords Generic.pkl


get the list od pickle files 

In [7]:
# get model list
forecasters_array = cuts_df.forecaster_name_list
forecasters_array

0               US Bing Ads Brand.pkl
1          US Bing Ads Competitor.pkl
2             US Bing Ads Generic.pkl
3         US Google AdWords Brand.pkl
4    US Google AdWords Competitor.pkl
5       US Google AdWords Generic.pkl
Name: forecaster_name_list, dtype: object

import the analytical file 

In [8]:
# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1) # read the analytical file .csv 
analytical_file['ds'] = pd.to_datetime(analytical_file['ds']) # ensure the date variable is in the correct datetime type
analytical_file_check_version = analytical_file.copy() # generate a copy 
analytical_file_check_version


,region,market,account_type,funnel,ds,X,Month,Year,US_Bing_Ads_Brand_Apr_2024,US_Bing_Ads_Brand_Jun_2024,...,US_Google_AdWords_Brand_May_2024,US_Google_AdWords_Competitor_Apr_2024,US_Google_AdWords_Competitor_Jun_2024,US_Google_AdWords_Competitor_May_2024,US_Google_AdWords_Generic_Apr_2024,US_Google_AdWords_Generic_Jun_2024,US_Google_AdWords_Generic_May_2024,Weekend,AOV,ROAS
0,US,US,Bing Ads,Brand,2024-04-25,3599.73,Apr,2024,3599.73,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,34.95,35.66
1,US,US,Bing Ads,Brand,2024-04-26,3307.80,Apr,2024,3307.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,34.95,35.66
2,US,US,Bing Ads,Brand,2024-04-27,957.14,Apr,2024,957.14,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,34.95,35.66
3,US,US,Bing Ads,Brand,2024-04-28,917.39,Apr,2024,917.39,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1,34.95,35.66
4,US,US,Bing Ads,Brand,2024-04-29,4319.26,Apr,2024,4319.26,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,34.95,35.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,US,US,Google AdWords,Generic,2024-06-26,500.58,Jun,2024,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,500.58,0.0,1,43.94,5.99
398,US,US,Google AdWords,Generic,2024-06-27,744.33,Jun,2024,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,744.33,0.0,1,43.94,5.99
399,US,US,Google AdWords,Generic,2024-06-28,362.71,Jun,2024,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,362.71,0.0,1,43.94,5.99
400,US,US,Google AdWords,Generic,2024-06-29,226.85,Jun,2024,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,226.85,0.0,1,43.94,5.99


import the constraint file 

In [9]:
# Get constrain file
constraint_file = pd.read_csv(constraint_file_name) # read the constraint file .csv
constraint_file['init'] = constraint_file[['MinWeight', 'MaxWeight']].mean(axis = 1) #create a variable with the initial wheights the mean between the min weight and the max weight
constraint_file

,market,account_type,funnel,Month,Year,variable_splits,Constrain_Variable_Name,periods_for_acc_cal,changepoint_prior_scale,seasonality_prior_scale,...,generic_cost_split,competitor_cost_split,new_confirmed,power,growth,periods,cutoff_month,MinWeight,MaxWeight,init
0,US,Bing Ads,Brand,Apr,2024,US_Bing_Ads_Brand_Apr_2024,cost,67,0.050,0.01,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
1,US,Bing Ads,Brand,May,2024,US_Bing_Ads_Brand_May_2024,cost,67,0.050,0.01,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
2,US,Bing Ads,Brand,Jun,2024,US_Bing_Ads_Brand_Jun_2024,cost,67,0.050,0.01,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
3,US,Bing Ads,Competitor,Apr,2024,US_Bing_Ads_Competitor_Apr_2024,cost,67,0.001,0.01,...,na,na,na,na,linear,67.0,2024-04-25,0.8,1.2,1.0
4,US,Bing Ads,Competitor,May,2024,US_Bing_Ads_Competitor_May_2024,cost,67,0.001,0.01,...,na,na,na,na,linear,67.0,2024-04-25,0.8,1.2,1.0
5,US,Bing Ads,Competitor,Jun,2024,US_Bing_Ads_Competitor_Jun_2024,cost,67,0.001,0.01,...,na,na,na,na,linear,67.0,2024-04-25,0.8,1.2,1.0
6,US,Bing Ads,Generic,Apr,2024,US_Bing_Ads_Generic_Apr_2024,cost,67,0.001,0.10,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
7,US,Bing Ads,Generic,May,2024,US_Bing_Ads_Generic_May_2024,cost,67,0.001,0.10,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
8,US,Bing Ads,Generic,Jun,2024,US_Bing_Ads_Generic_Jun_2024,cost,67,0.001,0.10,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0
9,US,Google AdWords,Brand,Apr,2024,US_Google_AdWords_Brand_Apr_2024,cost,67,0.100,0.10,...,na,na,na,na,logistic,67.0,2024-04-25,0.8,1.2,1.0


# PSO Function

Initialize the SavantPSO class function with the cuts, analytical and constraint file, also include the config dict

In [10]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)

Set the PSO hyper parameters 

In [11]:
itNum = 50 #entre 100 a 150 cuando se tiene mas iteraciones
n_particles = 0 #asi dentro de la columna me calcula el número de particulas optima
options = {'c1': 0.20255337412386598,
 'c2': 0.034812002156869526,
 'w': 0.8707709059066462}

optimize the budget allocation setting optimized input weights that maximize conversion quantity. <br>
    run_pso method of SavantPSO class function, with PSO hyper parameters

In [ ]:
result = resultPSO.run_pso(options, n_particles=n_particles, itNum = itNum)

2024-05-10 19:47:18,005 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.20255337412386598, 'c2': 0.034812002156869526, 'w': 0.8707709059066462}


/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/us/June 2024 Update/Input Weight History 05.10.2024.csv


pyswarms.single.global_best:   0%|          |0/50

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1000652.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1000652.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1000652.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1000652.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned

pyswarms.single.global_best:   2%|▏         |1/50, best_cost=-1e+6

...

yhat: -1001008.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1003046.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1001360.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -999777.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1002060.0
client_budget - analytical_spend_latest_period: 

pyswarms.single.global_best:  98%|█████████▊|49/50, best_cost=-1.02e+6

yhat: -1010740.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -998299.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1006547.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1007316.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1701650.0 difference between planned and forecast X:  0.0
yhat: -1000032.0
client_budget - analytical_spend_latest_period: 

pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-1.02e+6
2024-05-10 22:14:22,581 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1016822.0, best pos: [0.84911082 0.935801   0.84936147 0.80289543 1.15199269 1.04775805
 0.82663465 1.19867458 0.94712152 1.14328823 1.18352269 1.16278499
 1.07423181 0.84287318 0.8283733  1.09428186 1.01695739 0.81095619]


yhat: -998795.0
client_budget - analytical_spend_latest_period: 0
Compute duration: 8824.576738357544


Save the PSO optimization results and plot the cost through the different iterations

In [12]:
resultPSO.SaveResult(result)

NameError: name 'result' is not defined

#### if PSO optimization has already being run and we have some inititial input wehights

Read initial position input weights 

In [ ]:
pso_date = '01.21.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {pso_date}.csv' 
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1) # read the pso result file to get the input weights
inputWeights =np.tile(pos_df.pos.to_numpy(), [n_particles, 1]) # create the input weights parameters


2024-04-18 03:50:53,381 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.20255337412386598, 'c2': 0.034812002156869526, 'w': 0.8707709059066462}


/home/jupyter/All Regions/us/test of Juan/Input Weight History 04.18.2024.csv


pyswarms.single.global_best:   0%|          |0/50

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -627223.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -627223.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -627223.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -627223.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---

pyswarms.single.global_best:   2%|▏         |1/50, best_cost=-6.27e+5

...

yhat: -629150.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  -0.0
yhat: -628904.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -626296.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -627112.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -625284.0
client_budget - analytical_spend_latest_period: 0
-

pyswarms.single.global_best:  98%|█████████▊|49/50, best_cost=-6.32e+5

yhat: -627822.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -630134.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -626944.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -629498.0
client_budget - analytical_spend_latest_period: 0
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: -629088.0
client_budget - analytical_spend_latest_period: 0
--

pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-6.32e+5
2024-04-18 05:26:34,175 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -631591.0, best pos: [0.94147144 1.13235445 0.90735946 1.12861693 1.0601695  1.05763392
 1.18663699 1.14510455 0.99752745 0.80679573 1.01964194 0.86363084]


yhat: -628250.0
client_budget - analytical_spend_latest_period: 0
Compute duration: 5740.795632123947


run_pso method of SavantPSO class function, with PSO hyper parameters and input weights as initial position

In [ ]:
result = resultPSO.run_pso(options, n_particles=n_particles, itNum = itNum , init_pos=inputWeights)


if needed cave the PSO optimization results and plot the cost through the different iterations 

In [ ]:
resultPSO.SaveResult(result)